## Лабораторная работа №2 (курс "Математические методы анализа текстов")


#### Тема: Языковое моделирование и определение языка.


**Выдана**:   13 марта 2017

**Дедлайн**:   <font color='red'>9:00 утра 26 марта 2017</font>

**Среда выполнения**: Jupyter Notebook (Python 2.7)

#### Правила:

Результат выполнения задания $-$ отчет в формате Jupyter Notebook с кодом и выводами. В ходе выполнения задания требуется реализовать все необходимые алгоритмы, провести эксперименты и ответить на поставленные вопросы. Дополнительные выводы приветствуются. Чем меньше кода и больше комментариев $-$ тем лучше.

Все ячейки должны быть "выполненными", при этом результат должен воспроизвдиться при проверке (на Python 2.7). Если какой-то код не был запущен или отрабатывает с ошибками, то пункт не засчитывается. Задание, сданное после дедлайна, _не принимается_. Совсем.


Задание выполняется самостоятельно. Вы можете обсуждать идеи, объяснять друг другу материал, но не можете обмениваться частями своего кода. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов, а также предвзято негативное отношение семинаристов в будущем. Если вы нашли в Интернете какой-то код, который собираетесь заимствовать, обязательно укажите это в задании: вполне вероятно, что вы не единственный, кто найдёт и использует эту информацию.

#### Постановка задачи:

В данной лабораторной работе Вам предстоит реализовать n-грамную языковую модель с несколькими видами сглаживания:
- Add-one smoothing
- Stupid backoff
- Interpolation smoothing
- Kneser-Ney smoothing

Вы обучите ее на готовых корпусах, оцените качество и проведете ряд экспериментов. Во второй части задания Вы примените реализованную модель (но с буквенными n-граммами) к задаче распознавания языка. 


# Языковые модели

Цель языкового моделирования заключается в том, чтобы присвоить некоторые вероятности предложениям. Задача состоит в подсчете вероятности $P(W) = P(w_1, \dots, w_n)$ или $P(w_n \mid w_1, \dots, w_{n-1})$. Модель, умеющая вычислять хотя бы одну из этих двух вероятностей, называется **языковой моделью** (LM от Language Model).

Согласно **цепному правилу** (chain rule):

$$P(X_1, \dots, X_n) = P(X_1)P(X_2 \mid X_1)\dots P(X_n \mid X_1, \dots, X_{n-1}).$$ 

Также мы знаем, что

$$
    P(X_n \mid X_1, \dots, X_{n-1}) = \frac{P(X_1, \dots, X_n)}{P(X_1, \dots, X_{n-1})},
$$

следовательно, для того чтобы оценить $P(X_n \mid X_1, \dots, X_{n-1})$ нужно посчитать $P(X_1, \dots, X_n)$ и $P(X_1, \dots, X_{n-1})$. Но эти вероятности будут чрезвычайно малы, если мы возьмем большое $n$, так множество предложений из $n$ слов растет экспоненциально. Для упрощения применим **марковское предположение**: 

$$P(X_n \mid X_1, \dots, X_{n-1}) = P(X_n \mid X_{n - k + 1}, \dots, X_{n-1})$$

для некоторого фиксированного (небольшого) $k$. Это предположение говорит о том, что $X_{n}$ не зависит от $X_{1}, \dots, X_{n - k}$, то есть на следующее слово влияет лишь контекст из предыдущих $k - 1$ слова. Таким образом, мы получаем финальную вероятность:

$$
    P(w_1, \dots, w_n) = \prod_i P(w_i \mid w_{i-k+1}, \dots, w_{i - 1}).
$$

Далее для краткости будем обозначать $w_{i-k}^i := w_{i-k}, \dots, w_{i}$.

## Хранилище n-грам

Для начала выполним вспомогательную работу. Следуйте комментариям, чтобы написать NGramStorage с удобным интерфейсом.

In [3]:
import re
import math
import random
from collections import Counter
import numpy as np
from sklearn.cross_validation import train_test_split

/home/ruslan/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
class NGramIterator() :
    def __init__(self, sentence, n) :
        self.sent = sentence
        self.len_sent = len(sentence)
        self.left = 0
        self.right = n
        self.n = n
    
    def __iter__(self) :
        return self
    
    def __next__(self) :        
        if self.right <= self.len_sent:
            ngram = tuple(self.sent[self.left : self.right])
            self.left += 1
            self.right += 1
            return ngram
        raise StopIteration       

In [5]:
class NGramStorage:
    """Storage for ngrams' frequencies.
    
    Args:
        sents (list[list[str]]): List of sentences from which ngram
            frequencies are extracted.
        max_n (int): Upper bound of the length of ngrams.
            For instance if max_n = 2, then storage will store
            0, 1, 2-grams.
            
    Attributes:
        max_n (Readonly(int)): Upper bound of the length of ngrams.
    """
        
    def __init__(self, sents=[], max_n=0):
        self.__max_n = max_n
        self.__ngrams = {i: Counter() for i in range(self.__max_n + 1)}
        # self._ngrams[K] should have the following interface:
        # self._ngrams[K][(w_1, ..., w_K)] = number of times w_1, ..., w_K occured in words
        # self._ngrams[0][()] = number of all words
        
        ### YOUR CODE HERE
        for len_ngram in range(max_n + 1) :
            if len_ngram == 0 :
                self.__ngrams[len_ngram][()] += sum([len(sent) for sent in sents])
                continue
            for sent in sents :    
                for tup in NGramIterator(sent, len_ngram) :
                    self.__ngrams[len_ngram][tup] += 1
        ### END YOUR CODE
        
    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        if self.__max_n == 0 or u'UNK' in self.__ngrams[1]:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][(u'UNK',)] = 1
        
    @property
    def max_n(self):
        """Get max_n"""
        return self.__max_n
        
    def __getitem__(self, k):
        """Get dictionary of k-gram frequencies.
        
        Args:
            k (int): length of returning ngrams' frequencies.
            
        Returns:
            Dictionary (in fact Counter) of k-gram frequencies.
        """
        # Cheking the input
        if not isinstance(k, int):
            raise TypeError('k (length of ngrams) must be an integer!')
        if k > self.__max_n:
            raise ValueError('k (length of ngrams) must be less or equal to the maximal length!')
        return self.__ngrams[k]
    
    def __call__(self, ngram):
        """Return frequency of a given ngram.
        
        Args:
            ngram (tuple): ngram for which frequency should be computed.
            
        Returns:
            Frequency (int) of a given ngram.
        """
        # Cheking the input
        if not isinstance(ngram, tuple):
            raise TypeError('ngram must be a tuple!')
        if len(ngram) > self.__max_n:
            raise ValueError('length of ngram must be less or equal to the maximal length!')
        if len(ngram) == 1 and ngram not in self.__ngrams[1]:
            return self.__ngrams[1][(u'UNK', )]
        return self.__ngrams[len(ngram)][ngram]

## Оценка качества

Скачайте brown корпус, обучите модель и протестируйте на нескольких примерах последовательностей.

In [6]:
import nltk
# Uncomment next row and download brown corpus
# nltk.download()
from nltk.corpus import brown

In [22]:
all_sents = list(brown.sents())
random.shuffle(all_sents)
print('Number of all sentences = {}'.format(len(all_sents)))
train_sents = all_sents[:int(0.8 * len(all_sents))]
test_sents = all_sents[int(0.8 * len(all_sents)):]
print('Number of train sentences = {}'.format(len(train_sents)))
print('Number of test sentences = {}'.format(len(test_sents)))

Number of all sentences = 57340
Number of train sentences = 45872
Number of test sentences = 11468


In [18]:
# Create storage of 0, 1, 2, 3-grams
storage = NGramStorage(train_sents, 3)

In [19]:
# It's time to test your code
print(storage(('to', 'be')))
print(storage(('or',)))
print(storage(('not', 'to', 'be')))
print(storage(('somethingweird',)))
print(storage(()))

1478
3738
31
0
979646


Для численного измерения качества языковой модели определим **перплексию**:

$$
    {\mathbb{P}}(w_1, \dots, w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_i P(w_i \mid w_{i - k}, \dots, w_{i - 1})\right)^{-\frac1N},
$$

Вижно, что минимизация перплексии эквивалентна максимизации правдоподобия модели.

Реализуйте функцию по подсчету перплексии. Обратите внимание, что перплексия по корпусу равна произведению вероятностей **всех** предложений в степени $-\frac1N$, где $N -$ суммарная длина всех предложений.

In [10]:
def perplexity(estimator, sents):
    '''Estimate perplexity of the sequence of words using prob_estimator.'''
    ### YOUR CODE HERE
    perp  = [estimator.prob(sent) for sent in sents]
    perp = list(map(lambda t : np.log(t)if t > 1e-50 else np.log(1e-50), perp))
    perp = sum(perp)
    N = sum([len(sent) for sent in sents])
    perp = np.exp(-perp / N)
    ### END YOUR CODE
    
    return perp

## Оценка вероятностей n-грам

Первый и простейший способ оценки вероятностей N-грам следующий:

$$
    \hat P_{S}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N)}{c(w_1^{N-1})}.
$$

где $c(w_1^N)$ — это число последовательностей $w_1, \dots, w_N$ в корпусе, $S$ символизирует Straightforward. 

In [312]:
class StraightforwardProbabilityEstimator:
    """Class for simplest probability estimations of type P(word | context).
    
    P(word | context) = c(context + word) / c(context), where
    c(sequence) - number of occurances of the sequence in the corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
    """
    
    def __init__(self, storage):
        self.__storage = storage
        # Adding UNK token to avoid zero probabilities
        self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * phrase_counts / context_counts
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [313]:
# Initialize estimator
simple_estimator = StraightforwardProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(simple_estimator, test_sents)))
print(simple_estimator.prob('To be'.split()))
print(simple_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 250.40428137782538
1.6152079364857165e-05
0.0


Посчитаем перплексию униграмной модели.

In [314]:
uni_storage = NGramStorage(train_sents, 1)
uni_simple_estimator = StraightforwardProbabilityEstimator(uni_storage)
print('Simple estimator perplexity = {}'.format(perplexity(uni_simple_estimator, test_sents)))

Simple estimator perplexity = 103.13345578563413


Ответьте на следующие вопросы (внутри ipython ноутбука):

**Q:** Какие выводы можно сделать? Почему $P(\text{To be or not to be}) = 0$, хотя мы и добавили UNK токен?  
**A:** $P(\text{To be or not to be}) = 0$ потому что при подсчете пероятности предложения перемножаются вероятности вида $P(w^n|w^{n-1}_{n-k+1})$, а они уже могут быть равными 0, так как добавление UNK токена делает ненулевыми только вероятности вида $P(w^n)$

**Q:** Почему перплексия униграмной модели меньше, чем триграмной?  
**A:** Перплексия по сути является обратной величиной к среднему значению вероятности слова в предложении (в данном случае среднее геометрическое). В униграмной модели используется вероянтость слова без контекста, а в триграммной с контекстом. Когда униграммная модель видит слово, она выдаст просто частоту его вхождения в текст (если уже встречала его), а триграммная будет смотреть не только на слово, но и на контекст и если видит слово в незнакомом контексте, то даст ему маленькую вероятность, в отличии от униграммной мрдели. Думаю, что если бы корпус был больше, триграммная модель лучше бы познакомилась с текстом и на тесте выдала бы перплексию меньше, а на униграммную модель скорее всего увеличение корпуса не сильно повлияет, потому что она просто запоминает частоту вхождения слова.


## Add-one smoothing

Простейший вид сглаживания — **сглаживание Лапласа**. Чтобы избавиться от нулевых вероятностей $P(w_{N} \mid w_1^{N - 1})$, будем использовать формулу:

$$
    \hat P_{AOS}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N) + \delta}{c(w_1^{N-1}) + \delta V},
$$

где $V$ — это размер словаря, а $\delta$ — некоторая фиксированная константа.

Реализуйте класс, осуществляющий сглаживание Лапласа. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [315]:
class LaplaceProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = (c(context + word) + delta) / (c(context) + delta * V), where
    c(sequence) - number of occurances of the sequence in the corpus,
    delta - some constant,
    V - number of different words in corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): Smoothing parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        self.__V = len(set(storage[1].keys()))
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('context must be a tuple!')
            
        ### YOUR CODE HERE
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * (phrase_counts + self.__delta) / (context_counts + self.__delta * self.__V)
        prob = 1.
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

Подберите наилучший параметр $\delta$ для данного корпуса.

In [316]:
# Try to find out best delta parametr. We will not provide you any strater code.
### YOUR CODE HERE
best_delta = 1.
laplace_estimator = LaplaceProbabilityEstimator(storage, best_delta)
best = perplexity(laplace_estimator, test_sents)
for delta in range(2,10) :
    laplace_estimator = LaplaceProbabilityEstimator(storage, delta)
    perp = perplexity(laplace_estimator, test_sents)
    if perp < best :
        best_delta = delta
        best = perp
print('Best delta = ', best_delta)
print('Laplace estimator perplexity = {}'.format(best))
### END YOUR CODE

Best delta =  1.0
Laplace estimator perplexity = 227.54912307840942


Перебирал значения $\delta$, как целое число в диапазоне [1..9]

## Stupid backoff

Идея **простого отката** довольно понятна. Если у нас есть достаточно информцаии для подсчета вероятности $k$-грам, то будем использовать $k$-грамы. Иначе будем использовать вероятности $(k-1)$-грам с некоторым множителем, например, $0.4$, и так далее. К сожалению, в данном случае мы получим не вероятностное распределение, но в большинстве задач это не имеет принципиального значения. Если это все же важно, то необходимо подобрать множитель соответствующим образом.

Реализуйте класс, симулирующий сглаживание простым откатом. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [317]:
class StupidBackoffProbabilityEstimator:
    """Class for stupid backoff probability estimations.
    
    P(word | context) =
        P'(word | context),                  if  P'(word | context) > 0;
        P'(word | context[1:]) * multiplier, if  P'(word | context) == 0
                                             and P'(word | context[1:]) > 0;
        ...
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        multiplier (float): Multiplier which is used for probability estimations.
    """
    
    def __init__(self, base_estimator, multiplier=0.1):
        self.__base_estimator = base_estimator
        self.__mult = multiplier
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = self.__base_estimator(word, context)
        while prob == 0 and len(context) > 1 : 
            context = context[1:]
            prob = self.__base_estimator(word, context) * self.__mult
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [318]:
# Initialize estimator
sbackoff_estimator = StupidBackoffProbabilityEstimator(simple_estimator, .4)

# Let's make some estimations
print('Stupid backoff estimator perplexity = {}'.format(perplexity(sbackoff_estimator, test_sents)))
print(sbackoff_estimator.prob('To be'.split()))

Stupid backoff estimator perplexity = 213.40770910877336
1.6152079364857165e-05


Ответьте на следующие вопросы (внутри ipython ноутбука):

**Q:** Почему бессмысленно измерять перплексию в случае **Stupid backoff**?  
**A:** Как было сказано выше при использовании **Stupid backoff**, мы не получаем вероятностного распределения. При подсчете перплексии подразумевается, что мы все-таки считаем вероятности каждого предложния.


## Interpolation smoothing

В данном случае идея сглаживания посредством **интерполяции** также крайне проста. Пусть у нас есть $N$-грамная модель. Заведем вектор $\bar\lambda = (\lambda_1, \dots, \lambda_N)$, такой, что $\sum_i\lambda_i = 1$ и $\lambda_i \geq 0$. Тогда

$$
    \hat P_{IS}(w_{N} \mid w_1^{N-1}) = \sum_{i=1}^N \lambda_i \hat P_{S}(w_N \mid w_{N-i+1}^{N-1}).
$$

Придумайте, как обойтись одним вектором $\bar\lambda$, т.е. пользоваться им как в случае контекста длины $N$, так и при контексте меньшей длины (например, в начале предложения). Если мы просто обрубим сумму, то у нас уже не будет вероятностное распределение, что, конечно же, плохо.

In [319]:
class InterpolationProbabilityEstimator:
    """Class for interpolation probability estimations.
    
    P(word | context) =
        lambda_N * P'(word | context) +
        lambda_{N-1} * P'(word | context[1:]) +
        ... +
        lambda_1 * P'(word)
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        lambdas (np.array[float]): Lambdas which are used for probability estimations.
    """
    
    def __init__(self, base_estimator, lambdas):
        self.lambdas = lambdas
        self.__base_estimator = base_estimator
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = 0
        last_lambda = 0
        for i in range(len(self.lambdas)) :
            prob += self.lambdas[i] * self.__base_estimator(word, context)
            last_lambda += 1
            context = context[1:]
            if not context :
                break
            prob /= sum(self.lambdas[:last_lambda])
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [320]:
import numpy as np

# Initialize estimator
interpol_estimator = InterpolationProbabilityEstimator(simple_estimator, np.array([0.2, 0.2, 0.6]))

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(interpol_estimator, test_sents)))
print(interpol_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 237.37589652381666
6.46083174594e-07


Если контекст слишком маленький, вероятность вычислялась как взвешенная сумма вероятностей, пока контекст не пуст, но запоминался номер лямбда, до которого мы согли дойти, далее вероятность перенормировалась в зависимости от количества использованых лямбда. 

Обучить значения параметров $\lambda$ можно с помощью EM-алгоритма, но мы не будем этого здесь делать.

## Kneser-Ney smoothing

Идея данного сглаживания заключается в том, что словам, которые участвуют в большом количестве контекстов, присваиваются большие вероятности, а те, которые используются в паре-тройке контекстов, получают маленькие вероятности. Авторы данного сглаживания формализовали это следующим образом. Введем обозначения

$$
    N_{c}(w) := \left|\{\hat w : c(\hat w, w) > 0\}\right|
$$

$-$ число N-грамм, в которых последней частью идёт $w$ (слово или последовательность слов).

Опеределим рекурентное соотношение:

$$
    \hat P_{KN} (w_1) = \frac{N_{c}(w_1)}{\sum_{w} N_{c}(w)},
$$

$$
    \hat P_{KN}(w_{i} \mid w_{i - n + 1}^{i-1}) = \frac{{\rm max}\{c(w_{i -n +1}^i) - \delta, 0\}}{\sum_{w}c(w_{i - n + 1}^{i-1}, w)} + \lambda(w^{i-1}_{i-n+1}) \hat P_{KN}(w_{i} \mid w^{i-1}_{i-n+2}).
$$

где

$$
\lambda(w^{i-1}_{i-n+1}) = \frac{\delta}{\sum_{w}c(w_{i - n + 1}^{i-1}, w)}N_{c}(w_{i-n+1}^{i-1})
$$

$-$ весовой множитель.


Реализуйте данный подход.

In [23]:
class KneserNeyProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = ...
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): KneserNey parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        self.__ends = {i : Counter() for i in range(1, storage.max_n + 1)}
        self.__begins = {i : Counter() for i in range(1, storage.max_n + 1)}
        tuples = [storage[i].keys() for i in range(1, storage.max_n + 1)]
        for i in range(1, storage.max_n + 1) :
            for j in range(i, storage.max_n + 1) :
                for tup in tuples[j-1] :
                    self.__ends[i][tup[-i:]] += 1
                    self.__begins[i][tup[:i]] += 1
        
        self.__Nc_ones = sum(self.__ends[1].values())
    
    def get_ctr(self) : 
        return self.__ends, self.__begins
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
    
    def Nc(self, w, edge, count) :
        if edge :
            res = self.__ends[len(w)][w]
            if count :
                return res
            else :
                return 1 if res > 0 else 0
        else :
            res = self.__begins[len(w)][w]
            if count :
                return res
            else :
                return 1 if res > 0 else 0

                       
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        ### YOUR CODE HERE
        if not context :
            #print(self.__storage[1].keys())
            return self.Nc((word,), True, True)/ self.__Nc_ones
        else :
            sm = self.Nc(context, False, True)
            context_count = self.__storage(context + (word,))
            lmbda = self.__delta / sm * self.Nc(context, True, True) if sm > 0 else 0
            prob = max(context_count - self.__delta,0) / sm if sm > 0 else 0
            if lmbda == 0 :
                return prob
            else :
                return prob + lmbda * self(word, context[1:])                          
        ### END YOUR CODE
    
    def prob(self, sent): 
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [24]:
# Initialize estimator
kn_estimator = KneserNeyProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(kn_estimator, test_sents)))
print(kn_estimator.prob('To be'.split()))
print(kn_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 94.08931819304844
1.8744030413833268e-06
3.85219035855738e-13


При вычислении вероятности предложения, в случае когда $\sum_{w}c(w_{i - n + 1}^{i-1}, w) = 0$, занулял и $\lambda$ и текущее значение вероятности $\frac{{\rm max}\{c(w_{i -n +1}^i) - \delta, 0\}}{\sum_{w}c(w_{i - n + 1}^{i-1}, w)}$

Для вычисления значений вида $c(\hat w, w)$ завел вспомогательные счетчики вхождения $w$ в начало и в конец n-граммы

## Определение языка документа

**Постановка задачи:**  
Одна из задач, которая может быть решена при помощи языковых моделей $-$ **определение языка документа**. Реализуйте два классификатора для определения языка документа:
1. Наивный классификатор, который будет учитывать частоты символов и выбирать язык текста по признаку: распределение частот символов "наиболее похоже" на распределение частот символов в выбранном языке.
2. Классификатор на основе языковых моделей. Сами придумайте, как он должен работать.  
_Подсказка_: лучше считать n-грамы не по словам, а по символам.

---

**Как представлены данные:**  
Во всех текстовых файлах на каждой строчке записано отдельное предложение.
1. В папке _data_ находятся две папки: _full_ и _plain_. В _full_ находятся тексты в той форме, что они были взяты из сети, в _plain_ находятся те же самые тексты, но с них сначала была снята диакритика, а затем русский и греческий тексты были транслитерованы в английский.
2. В каждой из папок _full_ и _plain_ находятся папки _train_ и _test_.
3. В _train_ находятся файлы с текстами с говорящими именами, например, _ru.txt_, _en.txt_.
4. В _test_ находятся файлы _1.txt_, _2.txt_, $\dots$ в которых хранятся тексты, язык которых нужно определить. В этой же папке находится файл _ans.csv_, в котором вы можете найти правильные ответы и проверить, насколько хорошо сработали Ваши алгоритмы.

---

**Что нужно сделать:**  
Напишите два своих классификатора (которые описаны в постановке задачи) и получите максимально возможное accuracy на test-сете. Разрешается использовать только _train_ для обучения.

---

**В данном задании мы не предоставляем стартового кода!**

# 1. Наивный кассификатор
Завел хранилище униграмм. Для каждого языка отдельный счетчик вхождения символов. Классификация проводится следующим образом:
1. вычисляется частота вхождения символов для классифицируемого текста, далее значения нормируются. Получаем распределение над символами входящими в наш текст
2. затем подбирается язык, для которого KL дивергенция распределения его символов до рачпределения символов классифицируемого текста минимальна 

In [263]:
import os
cwd = os.getcwd()
path = cwd + '/language_detection/full/train/'
list_dir = os.listdir(path)

In [264]:
class UniStorage :
    """Storage for unigrams' frequencies.
    
    Args:
        filenames(list(str)) : list of filenames which contains labeled text
        path(str) : path to directory with texts
            
    Attributes:
        get_langs (list(str)): provided languages 
    """
    
    def __init__(self, filenames, path) :
        
        self.ugrams = {lang : Counter() for lang in [names[:2] for names in filenames]}
        self.langs_len = {lang : Counter() for lang in [names[:2] for names in filenames]}
        
        for name in filenames :
            file = open(path + '/' + name)
            try :
                for line in file :
                    for sym in line[:len(line) - 1] :
                        self.ugrams[str(name[:2])][sym] += 1
                        self.langs_len[str(name[:2])][()] += 1
            except UnicodeDecodeError:
                print(line)
                print(name)
            file.close()
    
    def __getitem__(self, lang):
        
        """Get dictionary of unigrams frequencies for lanuage lang.
        
        Args:
            lang(str): language
            
        Returns:
            Dictionary (in fact Counter) of unigrams frequencies for language lang.
        """

        if not isinstance(lang, str):
            raise TypeError('lang (language of dictionary) must be an string!')
        
        if not (lang in self.ugrams.keys()) :
            raise ValueError('this language is not finded!')
        
        return self.ugrams[lang]
    
    def get_langs(self) :
        
        """Get languages in storage"""
        return self.ugrams.keys()

In [265]:
class NaiveClassifier :
    
    """Class for predict language    
    
    Args:
        storage(UniStorage): Object of UniStorage class which will
            be used to extract frequencies of unigrams.
    """
    
    def __init__(self, storage):
        
        self.__storage = storage
        self.__langs = storage.get_langs()
    
    def add_one(self, lang_freq, sym) :
        """Smooth symbols frequency    

        Args:
            lang_freq(Counter): Object of Counter class which will
                be used to calculate frequencies of unigrams.
            sym(str) : Object of str class, current symbol цhich is checked 
                for entry to lang_freq. If lang_freq dosn't exist sym than
                return (1,1) - сompulsory occurrence
        """
        
        if sym in lang_freq.keys() :
            return (lang_freq[sym], 0)
        else :
            return (1, 1)
    
    def KL(self, p, l) :
        
        """KLdivergence which will be used as distance between 
            symbols frequencyes

        Args:
            p,l(Counter): Objects of Counter class which will
                be used to calculate frequencies of unigrams. 
                p - gold frequency of a fixed language. p belongs to self.__storage.
                l - current frequency for predict language
        """
        
        p, l = dict(p), dict(l)
        prob_space = set(p.keys()).union(set(l.keys()))
        dist = 0
        sum_l = sum(l.values())
        sum_p = sum(p.values())
        for sym in prob_space :
            p_i, new_p = self.add_one(p, sym)
            l_i, new_l = self.add_one(l, sym)
            sum_l += new_l
            sum_p += new_p
            dist += p_i * np.log(p_i / l_i)
        
        dist = 1 / sum_p * (dist + sum([self.add_one(p, sym)[0] * np.log(sum_l / sum_p) for sym in prob_space]))
        
        return dist
    
    def text2counter(self, path) :
        
        """Transforamation from text format to Counter(language frequancey) format

        Args:
            path(str): path to the text 
        """

        if not isinstance(path, str):
            raise TypeError('path must be a string!')
        
        file = open(path)
        countr = Counter()
        
        for line in file :
            for sym in line[:len(line) - 1] :
                countr[sym] += 1
        
        file.close()
        
        return countr
    
    def predict(self, countr) :
        
        """Predict language of text as language of the closest language frequancy 
            in storage by KLmteric

        Args:
            countr(Counter): Object of class Counter - language frequancy of 
            a predicted language
        """
        
        dists = {lang : 0 for lang in self.__langs}
        
        for lang in self.__langs :
            
            dists[lang] = self.KL(self.__storage[lang], countr)
        
        return min(dists, key = lambda i : dists[i])    

In [266]:
ustorage = UniStorage(list_dir, path)

In [267]:
clf = NaiveClassifier(ustorage)

In [268]:
ans = []
for num in range(1,241) :
    path = cwd + '/language_detection/full/test/' + str(num) + '.txt'
    ctr = clf.text2counter(path)
    ans.append(clf.predict(ctr))

In [269]:
import pandas as pd
gold_path = cwd + '/language_detection/full/test/ans.csv'
gold = pd.read_csv(gold_path, names = ['name', 'ans'])

In [270]:
gold_ans = list(gold['ans'])

In [278]:
from sklearn.metrics import accuracy_score
print('NaiveClassifier full accuracy = ', accuracy_score(ans, gold_ans))

NaiveClassifier full accuracy =  0.9875


In [274]:
path = cwd + '/language_detection/plain/train/'
list_dir = os.listdir(path)
list_dir.remove('.DS_Store')
ustorage_p = UniStorage(list_dir, path)

In [275]:
clf = NaiveClassifier(ustorage_p)

In [276]:
ans = []
for num in range(1,241) :
    path = cwd + '/language_detection/plain/test/' + str(num) + '.txt'
    ctr = clf.text2counter(path)
    ans.append(clf.predict(ctr))

In [277]:
gold_path = cwd + '/language_detection/full/test/ans.csv'
gold = pd.read_csv(gold_path, names = ['name', 'ans'])
gold_ans = list(gold['ans'])

In [279]:
print('NaiveClassifier plain accuracy = ', accuracy_score(ans, gold_ans))

NaiveClassifier plain accuracy =  0.9875


# 2 Классификатор на основе языковых моделей
1. Завел хранилище n - грамм и prob estimator.
2. Далее для каждого языка считаю перплексию входного текста
3. Выбираю язык, для которого перплексия оказалась минимальной (модель меньше остальных удивилась предоставленному тексту)

In [280]:
class MyNGramStorage :
    
    def __init__(self, max_n) :
        
        self.__max_n = max_n
        self.__ngrams = {i : Counter() for i in range(max_n + 1)}
    
    def update(self, sentence) :
        
        for len_ngram in range(self.__max_n + 1) :
            for tup in NGramIterator(sentence, len_ngram) :
                self.__ngrams[len_ngram][tup] += 1
    
    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        if self.__max_n == 0 or u'UNK' in self.__ngrams[1]:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][(u'UNK',)] = 1
    
    @property
    def max_n(self) :
        return self.__max_n
    
    def __getitem__(self, k) :
        
        return self.__ngrams[k]
    
    def __call__(self, ngram) :
        
        return self.__ngrams[len(ngram)][ngram]             

In [281]:
class MyNGramLangStorage :
    
    def __init__(self, filenames, direct, max_n) :
        
        self.__max_n = max_n
        self.__languages = set([name[:2] for name in filenames])
        self.__ngrams = {lang : MyNGramStorage(max_n) for lang in list(self.__languages)}
        
        for name in filenames :
            f = open(direct + name)
            for line in f :
                self.__ngrams[name[:2]].update(line[:len(line) - 1])
            f.close()
    
    def __getitem__(self, lang) :
        
        return self.__ngrams[lang]
    
    def languages(self) :
        return self.__languages

In [282]:
class MyEstimator:
    """Class for simplest probability estimations of type P(word | context).
    
    P(word | context) = c(context + word) / c(context), where
    c(sequence) - number of occurances of the sequence in the corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
    """
    
    def __init__(self, storage):
        self.__storage = storage
        # Adding UNK token to avoid zero probabilities
        self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * phrase_counts / context_counts
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob  

In [283]:
class MyClassifier :
    
    def __init__(self, lang_storage) :
        
        self.__estimators = {lang : MyEstimator(lang_storage[lang]) for lang in lang_storage.languages()}
        
    def perplexity(self, lang, text):
        '''Estimate perplexity of the sequence of words using prob_estimator.'''
        #perp  = sum([np.log(estimator.prob(sent)) if estimator.prob(sent) > 0 else -50 for sent in sents])
        #N = sum([len(sent) for sent in sents])
        #perp = np.exp(-perp / N)
        
        perp = 0
        N = 0
        estimator = self.__estimators[lang]
        for seq in text :
            words = seq[:len(seq) - 1].split(' ')
            N += len(words)
            perp += sum([np.log(estimator.prob(word)) if estimator.prob(word) > 0 else -50  for word in words])
        
        perp = np.exp(-perp / N)
        return perp
    
    def predict(self, path) :
        
        f = open(path)
        text = [line[:len(line) - 1] for line in f]
        f.close()
        langs = self.__estimators.keys()
        perps = {lang : 0 for lang in langs}
        
        for lang in langs :
            perps[lang] = self.perplexity(lang, text)
        
        return min(perps, key = lambda i : perps[i])
        
    
    def __call__(self, lang, word, context) :
        return self.__estimators[lang](word, context)
    
    def prob_sent(self, lang, sent) :
        
        return self.__estimators[lang].prob(sent)

In [287]:
path = cwd + '/language_detection/full/train/'
list_dir = os.listdir(path)

In [288]:
lang_store = MyNGramLangStorage(list_dir, path, 3)

In [289]:
clas = MyClassifier(lang_store)

In [290]:
ans = []
for num in range(1,241) :
    path = cwd + '/language_detection/full/test/' + str(num) + '.txt'
    ans.append(clas.predict(path))

In [291]:
import pandas as pd
gold_path = cwd + '/language_detection/full/test/ans.csv'
gold = pd.read_csv(gold_path, names = ['name', 'ans'])

In [292]:
gold_ans = list(gold['ans'])

In [293]:
from sklearn.metrics import accuracy_score
print('MyClassifier full accuracy = ', accuracy_score(ans, gold_ans))

MyClassifier full accuracy =  0.995833333333


In [294]:
path = cwd + '/language_detection/plain/train/'
list_dir = os.listdir(path)
list_dir.remove('.DS_Store')
lang_store_p = MyNGramLangStorage(list_dir, path, 3)

In [295]:
clas = MyClassifier(lang_store_p)

In [296]:
ans = []
for num in range(1,241) :
    path = cwd + '/language_detection/plain/test/' + str(num) + '.txt'
    ans.append(clas.predict(path))

In [297]:
gold_path = cwd + '/language_detection/full/test/ans.csv'
gold = pd.read_csv(gold_path, names = ['name', 'ans'])
gold_ans = list(gold['ans'])
print('MyClassifier full accuracy = ', accuracy_score(ans, gold_ans))

MyClassifier full accuracy =  1.0


Классификатор на основе языковой модели работает на много дольше, но показал при этом результат лучше чем наивный классификатор